# Loading import library

In [1]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem.Scaffolds.MurckoScaffold import MakeScaffoldGeneric
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles


from scripts import metrics
import importlib as imp
imp.reload(metrics)

<module 'scripts.metrics' from '/home/filv/PhD/github/recall_metrics/scripts/metrics.py'>

# Funktion for calculate metrics

In [3]:
def calculate_metrics(output_set, recall_set, number, scaffold_type, type_cluster, generator, receptor, save=True):

    mt = metrics.Metrics(type_cluster, scaffold_type, generator, number, receptor, save)     
    mt.load(output_set, recall_set)
    result = mt.calculate_metrics_with_return()
    display(result)
    return result

## Count metrics 
- Need the change parameters depend on which type of scaffold/cluster/generator/target you have

In [4]:
type_cluster = 'sim' #options: 'dis'|'sim' 
numbers = [0] #options: [0]|[0,1]|[0,1,2]|[0,1,2,3]|[0,1,2,3,4] -> all combinations 0-4, depends on which calculation you have
scaffold_type = 'csk' #options: 'csk'|'murcko'
generator = 'Molpher' #options: 'Molpher'|'DrugEx'|'REINVENT'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
save_options = True

In [5]:
for number in numbers:
    a0 = calculate_metrics(f"data/output_sets_new/{receptor}/cOS_{generator}_{type_cluster}_{number}_one_column.csv", \
                                                    f"data/input_recall_sets_new/{receptor}/cRS_{receptor}_{type_cluster}_{number}.csv",\
                       number, scaffold_type, type_cluster,generator,receptor,save_options)
    a0

data/output_sets_new/Glucocorticoid_receptor/cOS_Molpher_sim_0_one_column.csv
Convert_
Convert_
add_columns_same_like_input_function


/home/filv/PhD/github/recall_metrics/scripts/metrics.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

Save


,name,type_cluster,scaffold,uniq_scaffolds,set_size,TUPOR_,TUPOR,SESY,ASER,tRS
0,Molpher_0,sim,csk,109136,855964,29/38,0.763158,0.127501,0.014377,12306


## Calculate acarage value and save csv
- Do this step after calculate all metrics for each cluster

In [7]:
def avarage_value(number, type_cluster, scaffold_type, generator, receptor):

    path = f"data/results_new/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/"
    link = {}
    for x in range(number):
        link[f'link_{x}'] = f'{path}metrics_cluster_{x}_{type_cluster}_{generator}.csv'

    df = pd.DataFrame()
    for x,y in link.items():
        a = pd.read_csv(y)
        df = pd.concat([df, a], ignore_index=True)
        
    display(df)
    a = df.mean(numeric_only=True)
    df.loc[len(df)] = [f"{generator}_mean", type_cluster, scaffold_type, a['uniq_scaffolds'], a['set_size'], '-',\
                       a['TUPOR'], a['SESY'], a['ASER'], a['tRS']]
    df = df.round(7)
    dff = df.copy()
    dff['set_size'] = dff['set_size'].apply(lambda x : "{:,}".format(x))
    dff['uniq_scaffolds'] = dff['uniq_scaffolds'].apply(lambda x : "{:,}".format(x))
    dff['tRS'] = dff['tRS'].apply(lambda x : "{:,}".format(x))

    mean = pd.DataFrame()
    mean = df.loc[number].to_frame().T
                                        
    display(mean)
    dff.to_csv(f"data/results_new/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean_with_coma.csv", index = False)
    df.to_csv(f"data/results_new/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean.csv", index = False)
    dff.to_html(f"data/results_new/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean.html", index = False)
    mean.to_csv(f"data/results_new/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}_mean_{scaffold_type}_{type_cluster}.csv", index=False) #float_format='%.7f'

In [16]:
type_cluster = 'sim' #options: 'dis'|'sim' 
number = 5 #options: 1-5
scaffold_type = 'csk' #options: 'csk'|'murcko'
generator = 'Molpher' #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

In [17]:
avarage_value(number, type_cluster, scaffold_type, generator, receptor)

,name,type_cluster,scaffold,uniq_scaffolds,set_size,TUPOR_,TUPOR,SESY,ASER,tRS
0,Molpher_0,sim,csk,109136,855964,29/38,0.763158,0.127501,0.014377,12306
1,Molpher_1,sim,csk,102110,818233,28/38,0.736842,0.124793,0.020957,17148
2,Molpher_2,sim,csk,101071,804684,23/37,0.621622,0.125603,0.013162,10591
3,Molpher_3,sim,csk,98457,801639,26/35,0.742857,0.122820,0.012449,9980
4,Molpher_4,sim,csk,108264,901697,22/35,0.628571,0.120067,0.004182,3771


,name,type_cluster,scaffold,uniq_scaffolds,set_size,TUPOR_,TUPOR,SESY,ASER,tRS
5,Molpher_mean,sim,csk,103807.6,836443.4,-,0.69861,0.124157,0.013026,10759.2


# Connect all mean value to one pandas DataFrame

In [13]:
def connect_mean_value(reeptor, scaffold_type, type_cluster, generators_name_list):
    path = f"data/results_new/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/"
    link = {}
    for generator in generators_name_list:
        link[f'link_{generator}'] = f'{path}{generator}_mean_{scaffold_type}_{type_cluster}.csv'

    df = pd.DataFrame()
    for x,y in link.items():
        a = pd.read_csv(y)
        df = pd.concat([df, a], ignore_index=True)
    display(df)
    dff = df.copy()
    dff['set_size'] = dff['set_size'].apply(lambda x : "{:,}".format(x))
    dff['uniq_scaffolds'] = dff['uniq_scaffolds'].apply(lambda x : "{:,}".format(x))
    dff['tRS'] = dff['tRS'].apply(lambda x : "{:,}".format(x))
    
    dff.to_csv(f"{path}/mean_{scaffold_type}_{type_cluster}_with_coma.csv", index = False)
    df.to_csv(f"{path}/mean_{scaffold_type}_{type_cluster}.csv", index = False)
    dff.to_html(f"{path}/mean_{scaffold_type}_{type_cluster}.html", index = False)

In [14]:
type_cluster = 'dis' #options: 'dis'|'sim' 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generators_name_list = ['Molpher', 'DrugEx', 'DrugEx3', 'REINVENT'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

In [15]:
connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

FileNotFoundError: [Errno 2] No such file or directory: 'data/results_new/Glucocorticoid_receptor/csk_scaffolds/dis/DrugEx_mean_csk_dis.csv'

In [20]:
connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

,name,type_cluster,scaffold,uniq_scaffolds,set_size,TUPOR_,TUPOR,SESY,ASER,tRS
0,Molpher_mean,sim,csk,129157.6,1047941.4,-,0.678550,0.123292,0.011387,11821.0
1,DrugEx_mean,sim,csk,338794.8,999886.4,-,0.639048,0.338835,0.018002,18000.4
2,DrugEx3_mean,sim,csk,183183.8,999947.0,-,0.765563,0.183194,0.023718,23716.4
3,REINVENT_mean,sim,csk,158796.6,1057641.2,-,0.537013,0.150142,0.016103,17030.2
